In [3]:
# import needed libraries
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [167]:
def teamSeasonStats(years):
    
    teamPerGame_df = pd.DataFrame(columns = ["Year","Team","G","MP","FG","FGA","FG%","3P","3PA",
                                     "3P%","2P","2PA","2P%","FT","FTA","FT%","ORB","DRB","TRB",
                                     "AST","STL","BLK","TOV","PF","PTS"])
    
    teamAdvanced_df = pd.DataFrame(columns = ["Year","Team","Age","W","L","PW","PL","MOV","SOS","SRS","ORtg","DRtg",
                                              "NRtg","Pace","FTr","3PAr","TS%","eFG%","TOV%","ORB%","FT/FGA","eFG%",
                                              "TOV%","DRB%","FT/FGA","Arena","Attend.","Attend./G"])
    
    teamShooting_df = pd.DataFrame(columns = ["Year","Team","G","MP","FG%","Dist.","2P","0-3","3-10","10-16","16-3P",
                                              "3P","2P","0-3","3-10","10-16","16-3P","3P","2P","3P","%FGA","Md.","%FGA",
                                              "Md.","%3PA","3P%","Att.","Md."])
    
    dfs = [teamPerGame_df,teamAdvanced_df,teamShooting_df]
    
    # loop through each year
    for year in years:
        
        # URL to scrape, notice f string:
        url = f"https://www.basketball-reference.com/leagues/NBA_{year}.html"

        # collect HTML data
        html = urlopen(url)
        
        # create beautiful soup object from HTML
        soup = BeautifulSoup(html, features="lxml")
        
        if year <= 2015 and year > 2004:
            teamPerGame_headers = [th.getText() for th in soup.findAll('tr', limit=40)[39].findAll('th')]
            teamAdvanced_headers = [th.getText() for th in soup.findAll('tr', limit=231)[230].findAll('th')]
            teamShooting_headers = [th.getText() for th in soup.findAll('tr', limit=264)[263].findAll('th')]
        elif year <= 2004 and year > 1996:
            teamPerGame_headers = [th.getText() for th in soup.findAll('tr', limit=37)[36].findAll('th')]
            teamAdvanced_headers = [th.getText() for th in soup.findAll('tr', limit=222)[221].findAll('th')]
            teamShooting_headers = [th.getText() for th in soup.findAll('tr', limit=254)[253].findAll('th')]
        elif year > 2015:
            teamPerGame_headers = [th.getText() for th in soup.findAll('tr', limit=73)[72].findAll('th')]
            teamAdvanced_headers = [th.getText() for th in soup.findAll('tr', limit=264)[263].findAll('th')]
            teamShooting_headers = [th.getText() for th in soup.findAll('tr', limit=297)[296].findAll('th')]
        else:
            print(f"year{year} not in range")
            return
        
        headers = [teamPerGame_headers,teamAdvanced_headers,teamShooting_headers]
        for i in range(len(headers)):
            headers[i] = headers[i][1:]
            headers[i].insert(0, "Year")
            if year <= 2001 and i == 2:
                headers[i].insert(-5, "%FGA")
                headers[i].insert(-5, "Md.")

            while(True):
                try:
                    headers[i].remove('\xa0')
                except:
                    break
            
            
        # next, grab all data from rows (avoid first row)
        rows = soup.findAll('tr')
#         for i in range(len(rows)):
#             print(i)
#             print([td.getText() for td in rows[i].findAll('td')])
#         return
        if year <= 2015 and year > 2004:
            teamPerGame_rowData = [[td.getText() for td in rows[i].findAll('td')]
                        for i in range(40,72)]
            teamAdvanced_rowData = [[td.getText() for td in rows[i].findAll('td')]
                        for i in range(231,263)]
            teamShooting_rowData = [[td.getText() for td in rows[i].findAll('td')]
                        for i in range(264,296)]
        elif year <= 2004 and year > 1996:
            teamPerGame_rowData = [[td.getText() for td in rows[i].findAll('td')]
                        for i in range(37,68)]
            teamAdvanced_rowData = [[td.getText() for td in rows[i].findAll('td')]
                        for i in range(222,253)]
            teamShooting_rowData = [[td.getText() for td in rows[i].findAll('td')]
                        for i in range(254,285)]
        elif year > 2015:
            teamPerGame_rowData = [[td.getText() for td in rows[i].findAll('td')]
                        for i in range(73,105)]
            teamAdvanced_rowData = [[td.getText() for td in rows[i].findAll('td')]
                        for i in range(264,296)]
            teamShooting_rowData = [[td.getText() for td in rows[i].findAll('td')]
                        for i in range(297,329)]
        else:
            print(f"year{year} not in range")
            return
            
        # find first empty []
        teamPerGame_teamStats = []
        teamAdvanced_teamStats = []
        teamShooting_teamStats = []
        
        row_datas = [teamPerGame_rowData,teamAdvanced_rowData,teamShooting_rowData]
        team_stats = [teamPerGame_teamStats,teamAdvanced_teamStats,teamShooting_teamStats]
        
        for table in range(3):
            row_data = row_datas[table]
            team_stat = team_stats[table]
            for i in range(len(row_data)):
                if row_data[i] != []:
                    if (row_data[i][0][-1] == '*'):
                        row_data[i][0] = row_data[i][0][:-1]
                    team_stat += [row_data[i]]
                else:
                    break
            countEmpty=[]
            first=True
                        
            for i in range(0, len(team_stat)):
                team_stat[i].insert(0, year)
                if year <= 2000 and table == 1:
                    team_stat[i].insert(-1, 0)
                if year <= 2001 and table == 2:
                    team_stat[i].insert(-5, 0)
                    team_stat[i].insert(-5, 0)
                    
                if i == len(team_stat)-1:
                    for j in countEmpty:
                        team_stat[i].pop(j)
                    break;
                
                while(True):
                    try:
                        if first:
                            countEmpty.append(team_stat[i].index(''))
                        team_stat[i].remove('')

                    except:
                        first=False
                        break
                

#             print(team_stat)
            year_standings = pd.DataFrame(team_stat, columns = headers[table])
#             print(year_standings)
#             print(headers[table])
#             print(dfs[table])
            dfs[table] = dfs[table].append(year_standings,ignore_index=True)
    #end loop
    
    dfs[0].columns= ["Year","Team","G","MP","FG","FGA","FG%","3P","3PA",
                                     "3P%","2P","2PA","2P%","FT","FTA","FT%","ORB","DRB","TRB",
                                     "AST","STL","BLK","TOV","PF","PTS"]
    
    dfs[1].columns = ["Year","Team","Age","W","L","PW","PL","MOV","SOS","SRS","ORtg","DRtg",
                                              "NRtg","Pace","FTr","3PAr","TS%","eFG%","TOV%","ORB%","FT/FGA","oppo. eFG%",
                                              "oppo. TOV%","oppo. DRB%","oppo. FT/FGA","Arena","Attend.","Attend./G"]
    
    dfs[2].columns = ["Year","Team","G","MP","FG%","Dist.","%FGA 2P","%FGA 0-3","%FGA 3-10","%FGA 10-16","%FGA 16-3P",
                                              "%FGA 3P","%FG 2P","%FG 0-3","%FG 3-10","%FG 10-16","%FG 16-3P","%FG 3P","%Assist 2P","%Assist 3P",
                                              "dunk %FGA","dunk Md.","layup %FGA","layup Md.","corner %3PA","corner 3P%","heaves Att.","heaves Md."]
#     print(teamShooting_df)
    return dfs

In [173]:
# teamStats = teamSeasonStats([i for i in range(2000,2020)])
teamPerGame_df = teamStats[0]
teamAdvanced_df = teamStats[1]
teamShooting_df = teamStats[2] 
# print(teamShooting_df.head())

###DO NOT RUN. FILES HAVE BEEN CREATED
# teamPerGame_df.to_csv("TeamPerGameStats2000_2021.csv", index=False)
# teamAdvanced_df.to_csv("TeamAdvancedStats2000_2021.csv", index=False)
# teamShooting_df.to_csv("TeamShootingStats2000_2021.csv", index=False)


In [174]:
def MVPVoting(years):
    
    MVP_df = pd.DataFrame(columns = ["Year", "Player","Age","Tm","First","Pts Won","Pts Max"
                                     ,"Share","G","MP","PTS","TRB","AST","STL","BLK","FG%","3P%","FT%","WS","WS/48"])
    # loop through each year
    for year in years:
        
        # URL to scrape, notice f string:
        url = f"https://www.basketball-reference.com/awards/awards_{year}.html"

        # collect HTML data
        html = urlopen(url)
        
        # create beautiful soup object from HTML
        soup = BeautifulSoup(html, features="lxml")
        
#         print(soup)
        # use getText()to extract the headers into a list
        headers = [th.getText() for th in soup.findAll('tr', limit=2)[1].findAll('th')]
        headers = headers[1:]
        
#         print(headers)
        
        # next, grab all data from rows (avoid first row)
        rows = soup.findAll('tr')[2:]
        row_data = [[td.getText() for td in rows[i].findAll('td')]
                    for i in range(len(rows))]
        # find first empty []
        team_stats = []
        i = 0
        while(True):
            if row_data[i]!= []:
                team_stats += [row_data[i]]
            else:
                break
            i+=1
        
#         print(team_stats)

        # add year to each row in team_stats
        for i in range(0, len(team_stats)):
            team_stats[i].insert(0, year)
            
            
        # add year columns to headers
        headers.insert(0, "Year")
        
        # create a dataframe with all aquired info
        year_standings = pd.DataFrame(team_stats, columns = headers)
        MVP_df = MVP_df.append(year_standings)
#     # export to csv
    return MVP_df

In [175]:
# MVP = MVPVoting([i for i in range(2000,2022)])
print(MVP.head())

###DO NOT RUN. FILES HAVE BEEN CREATED
# MVP.to_csv("MVPdata2000_2021.csv", index=False)


   Year            Player Age   Tm  First Pts Won Pts Max  Share   G    MP  \
0  2000  Shaquille O'Neal  27  LAL  120.0  1207.0    1210  0.998  79  40.0   
1  2000     Kevin Garnett  23  MIN    0.0   408.0    1210  0.337  81  40.0   
2  2000   Alonzo Mourning  29  MIA    0.0   367.0    1210  0.303  79  34.8   
3  2000       Karl Malone  36  UTA    0.0   312.0    1210  0.258  82  35.9   
4  2000        Tim Duncan  23  SAS    0.0   248.0    1210  0.205  74  38.9   

    PTS   TRB  AST  STL  BLK   FG%   3P%   FT%    WS WS/48  
0  29.7  13.6  3.8  0.5  3.0  .574  .000  .524  18.6  .283  
1  22.9  11.8  5.0  1.5  1.6  .497  .370  .765  11.6  .172  
2  21.7   9.5  1.6  0.5  3.7  .551  .000  .711  12.9  .226  
3  25.5   9.5  3.7  1.0  0.9  .509  .250  .797  15.3  .249  
4  23.2  12.4  3.2  0.9  2.2  .490  .091  .761  13.0  .218  
